# Growth Rate Computation of MPAS Experiments

## import package

In [40]:
import sys
import numpy as np
import joblib as jl

sys.path.append('/home/b11209013/Package/')
import Theory as th # type: ignore
import DataProcess as dp # type: ignore
import SignalProcess as sp # type: ignore

## Functions

### Covariance

In [41]:
ft = sp.Fourier()

def Covariance(data1, data2):
    
    cs = ft.CrossSpectrum(data1, data2)

    cs_smooth = np.empty(cs.shape, dtype = complex)
    
    kernel = np.array([1, 2, 1]) / 4
    
    for i in range(cs.shape[0]):
        cs_smooth[i] = np.convolve(cs[i], kernel, mode='same')
    
    for i in range(cs.shape[1]):
        cs_smooth[:, i] = np.convolve(cs_smooth[:, i], kernel, mode='same')
    
    return cs_smooth

### Phase

In [42]:
def Phase(data1, data2):
    
    cs = np.array([
        Covariance(data1[i], data2[i])
        for i in range(data1.shape[0])
    ])
    
    phase = np.atan2(cs.imag, cs.real)
    
    return phase

### Growth Rate

In [43]:
def Growth_Rate(data1, data2):
    
    var = np.array([
        Covariance(data2[i], data2[i]).real
        for i in range(data2.shape[0])
    ]).mean(axis=0)
    cov = np.array([
        Covariance(data1[i], data2[i])
        for i in range(data2.shape[0])
    ]).mean(axis=0)
    
    sigma = 2*np.real(cov) / var
    
    return sigma

### Coherence Square

In [44]:
def Coherence(data1, data2):
    
    var1 = np.array([
        Covariance(data1[i], data1[i]).real
        for i in range(data1.shape[0])
    ]).mean(axis=0)
    var2 = np.array([
        Covariance(data2[i], data2[i]).real
        for i in range(data2.shape[0])
    ]).mean(axis=0)
    cov  = np.array([
        Covariance(data1[i], data2[i])
        for i in range(data1.shape[0])
    ]).mean(axis=0)
    
    Coh2 = ((cov.real)**2 + (cov.imag)**2) / (var1 * var2)

    return Coh2

## Load data

In [45]:
# case name
case: str = 'CNTL'

# path
fname: str = '/home/b11209013/2024_Research/MPAS/PCA_file/CNTL_PC.joblib'

# load data
data: dict = jl.load(fname)

# extract data
pc: dict[str: np.ndarray] = {}

lon : np.ndarray = data['lon']
lat : np.ndarray = data['lat']
time: np.ndarray = data['time']

pc['tpc1']  = data['pc1']['t']
pc['tpc2']  = data['pc2']['t']
pc['q1pc1'] = data['pc1']['q1']
pc['q1pc2'] = data['pc2']['q1']

## Process Data

### Format data

In [46]:
fmt = dp.Format(lat)

ps_sym = {
    var: fmt.sym(pc[var])
    for var in pc.keys()
}

### Windowing data

In [47]:
Hann_windowing = np.hanning(120)[:, None]

ps_hann = {
    var: np.array([
        ps_sym[var][i*60:i*60+120] * Hann_windowing
        for i in range(5)
        ])
    for var in pc.keys()
}


## Compute Variables}

In [48]:
phase = {
    'pc1': Phase(ps_hann['tpc1'], ps_hann['q1pc1']),
    'pc2': Phase(ps_hann['tpc2'], ps_hann['q1pc2'])
}

### Growth Rate

In [49]:
sigma = {
    'pc1': Growth_Rate(ps_hann['q1pc1'], ps_hann['tpc1']),
    'pc2': Growth_Rate(ps_hann['q1pc2'], ps_hann['tpc2']),
}

### Coherence Square

In [50]:
Coh2 = {
    'pc1': Coherence(ps_hann['q1pc1'], ps_hann['tpc1']),
    'pc2': Coherence(ps_hann['q1pc2'], ps_hann['tpc2']),
}

### Fishers' Z-transform

In [53]:
def z_transform(coh2):
    r = np.sqrt(coh2)

    return np.log((1+r)/(1-r))

Z_trans = {
    'pc1': z_transform(Coh2['pc1']),
    'pc2': z_transform(Coh2['pc2']),
}

## Statistics Test

### Z-test on Transformed Z

## Plot Growth Rate Diagram

### Plot setting

In [51]:
wn = np.linspace(-360, 360, 720)
fr = np.linspace(1/30, 2, 60)

wnm, frm = np.meshgrid(wn, fr)

### Growth Rate